<a href="https://colab.research.google.com/github/riacheruvu/automated-data-science/blob/master/Automated_Dataset_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated Dataset Selection

This is a preliminary, work in progress attempt to parse results from Google Data Search (https://toolbox.google.com/datasetsearch) using BeautifulSoup in response to a query ("business problem") and apply fuzzy string matching to the query in order to determine the appropriate dataset.

Please ignore the unclean code and comments; a thorough clean-up and revision of the implementation is coming soon.

# Import Necessary Packages

In [138]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import webbrowser
import requests
#from urllib.request import urlopen as uReq
import urllib
!pip install fuzzywuzzy
from bs4 import BeautifulSoup
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from ipywidgets import widgets
from IPython.display import display

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Read and analyze your request

Hi! This tool is meant for supervised learning applications (in other words, given some data, the machine learning algorithm will make predictions). It also only supports prediction of a single feature at the moment.

Answer a few questions about the type of business problem you're interested in, and we'll whip up some datasets you might be interested in.

In [313]:
#Code for Ipython widgets

from ipywidgets import widgets
from IPython.display import display
text = widgets.Text()
new_text = widgets.Text()
even_newer_text = widgets.Text()

type_of_dataset = input("What is the topic of the business problem would you like to solve? For example, try 'Boston Education Data'\n")

keywords = input("What keywords would you use to describe the dataset(s) that fits your application?")

What is the topic of the business problem would you like to solve? For example, try 'Boston Education Data'
Boston Education Data
What keywords would you use to describe the dataset(s) that fits your application?boston school facility yearly data building learning environments


In [0]:
stop_words = set(stopwords.words('english')) 
  
word_tokens = word_tokenize(type_of_dataset) 
  
filtered_sentence = [w for w in word_tokens if not w in stop_words] 

In [332]:
print(filtered_sentence)

['Boston', 'Education', 'Data']


# Find and download the right dataset for you

In [316]:
URL = 'https://toolbox.google.com/datasetsearch/search?query='

for i in range(len(filtered_sentence)):
  URL = URL + filtered_sentence[i] + '%20'
  
URL

'https://toolbox.google.com/datasetsearch/search?query=Boston%20Education%20Data%20'

In [0]:
data = requests.get(URL) #gets all the content of webpage
soup = BeautifulSoup(data.text,'html.parser')

In [0]:
hi = soup.prettify()
hi = soup.get_text()

In [0]:
hi = str(hi)
hi = hi.split("data:function(){return ",1)[1]
l = hi.replace('null','')
p = l.replace('\n', '')

In [0]:
import re
#Get Next Name of Dataset
compare = []

rest = p.split("'\\',", 1)[0]

In [0]:
rest = re.findall('"([^"]*)"', rest)

In [0]:
#Code reused from https://stackoverflow.com/questions/28050350/split-list-of-strings-into-list-of-sublists-based-on-a-string
questionchunks = []
hithere = []
qlist = []
for line in rest:

    if (line != '\\' ):
        questionchunks.append(line)      # add the element to each of your chunk   
    else: 
        qlist.append(questionchunks)   # append chunk
        questionchunks = []       # reset chunk

In [0]:
len(questionchunks)
if str(questionchunks).startswith( "[''") == True: ##Need to confirm this is true in all cases where questionchunks doesn't contain the real info
    flat_list = [item for sublist in qlist for item in sublist]
else:
  flat_list = questionchunks

In [327]:
real_list=[]
real_list.append([item.split() for item in ' '.join(flat_list).split('AAAAAA') if item])
c= 0
for x in real_list:
    for y in x:
        c = c +1
print(c)

real_list[0][0]

100


['Boston',
 'Education',
 'Data',
 'BuildBPS:',
 'Facilities',
 'and',
 'Educational',
 'Data',
 'for',
 'Boston',
 'Public',
 'Schools',
 'data.boston.gov',
 'Analyze',
 'Boston',
 'https://data.boston.gov',
 'boston.gov',
 'https://data.boston.gov/dataset/buildbps-facilities-and-educational-data-for-boston-public-schools',
 'BuildBPS',
 'Buildings',
 'Facilities',
 'Operations',
 'Schools',
 'xlsx',
 'pdf',
 'csv',
 'http://www.opendefinition.org/licenses/odc-pddl',
 'Apr',
 '5,',
 '2017',
 'May',
 '14,',
 '2019',
 'Boston',
 'Public',
 'Schools',
 'http://www.bostonpublicschools.org/',
 'https://t2.gstatic.com/images?q\\u003dtbn:ANd9GcSithRu5WIlgCsrUECZZmtY8agqshFNdkwwx9vqdb_SAsqnShFW',
 '\\u003cp',
 'class\\u003d\\',
 '\\u003eBuildBPS',
 'compiles',
 'vast',
 'amounts',
 'of',
 'data',
 'and',
 'allows',
 'for',
 'ongoing',
 'analyses',
 'that',
 'can',
 'be',
 'used',
 'to',
 'guide',
 'and',
 'inform',
 'decisions',
 'related',
 'to',
 'school',
 'building',
 'investments.',
 'An

In [328]:
fuzzy_scores = {}
for pi in range(0,10):
    print(pi)
    result = (real_list[0][pi])
    hello = []

    preprocessed_text = re.sub(r'.*\u003e', '\u003e', str(result))
    
    hello.append(preprocessed_text)

    hi = str(hello).replace("\\", "")
    bye = hi.replace("/br", " ")
    bye.replace("/p", " ")
    rep = {"src": "", "alt": 
           "", "a href": "", "pnnp": "", "class": "", "/a": "", "/em": "", "/p": "", "img": "", 
           "boqResearchsciencesearchdesktopuiDetailDescriptionElement_": "",  "blockquote": "", "u003e": "", "u003d" : "", "u003c": "", "\\": ""} 

    rep = dict((re.escape(k), v) for k, v in rep.items())
    pattern = re.compile("|".join(rep.keys()))
    text = pattern.sub(lambda m: rep[re.escape(m.group(0))], bye)
    best = re.sub('07pwN8NZpOr9gbTfAAAAAA', '', text) 
    new_list = str(best).replace('" ' '",', "")
    newer_list = str(new_list).replace('\'', "")
    another_list = str(newer_list).replace(',', "")
    another_new_list = str(another_list).replace('"', "")
    string = another_new_list.replace('[','').replace(']','')
    text = re.sub(r'@\S+', 'Someone', string)
    text = text.replace("title", " ")
    pattern = re.compile(r"\d*([^\d\W]+)\d*")
    text = re.sub(r'http\S+', '', text)
    text = pattern.sub(r"\1", text)
    text = text.replace("  ", " ")
    text = text.rsplit(' ', 1)[0]
    x = [str(i) for i in text.split()]
    for word in x:
        if word.endswith(".txt") == True:
            text = text.replace(word, " ")
        if word.endswith(".csv") == True:
            text = text.replace(word, " ")
        if word.endswith(".gov") == True:
            text = text.replace(word, " ")
        if word.endswith(".py") == True:
            text = text.replace(word, " ")
        if word.endswith(".com") == True:
            text = text.replace(word, " ")
        if word.endswith(".org") == True:
            text = text.replace(word, " ")
        if word.endswith(".edu") == True:
            text = text.replace(word, " ")
    print(text)
    stop_words = set(stopwords.words('english')) 

    word_tokens = word_tokenize(text) 

    filtered_sentence = [w for w in word_tokens if not w in stop_words] 

    filtered_sentence = [] 

    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    
    fuzzy_scores.update( {str(filtered_sentence[:10]) : fuzz.token_set_ratio(filtered_sentence, keywords)} )

0
Boston Education Data BuildBPS: Facilities and Educational Data for Boston Public Schools   Analyze Boston   BuildBPS Buildings Facilities Operations Schools xlsx pdf csv Apr 5 2017 May 14 2019 Boston Public Schools  p BuildBPS compiles vast amounts of data and allows for ongoing analyses that can be used to guide and inform decisions related to school building investments. An assessment team of architects and educational planners visited all of Boston Public Schools’ buildings collecting and organizing information on the physical condition of each building and their educational suitability to the programs offered within. Beyond the general information about each school building the data is organized in the following four categories: Facility Assessment– Building Facility Assessment–Site Educational Facility Effectiveness: Learning Environments and Educational Facility Effectiveness: Spaces. This dataset will be updated on a yearly basis to reflect changes as renovations and upgrades

**The final text parsed from one of the results returned by Google Data Search:  **

In [329]:
fuzzy_scores

{"['All', 'Employees', ':', 'Education', 'Heh', 'Services', 'Boston-Cambridge-Nashua', 'MA-NH', '(', 'NECTA']": 29,
 "['All', 'Employees', ':', 'Education', 'Heh', 'Services', 'Boston-Cambridge-Newton', 'MA', '(', 'NECTA']": 29,
 "['All', 'Employees', ':', 'Educational', 'Services', 'Boston-Cambridge-Newton', 'MA', '(', 'NECTA', 'Division']": 29,
 "['All', 'Employees', ':', 'State', 'Government', 'Educational', 'Services', 'Boston-Cambridge-Newton', 'MA', '(']": 29,
 "['All', 'Employees', ':', 'State', 'Government', 'Educational', 'Services', 'Boston-Cambridge-Quincy', 'MA-NH', '(']": 29,
 "['Boston', 'Education', 'Data', 'BuildBPS', ':', 'Facilities', 'Educational', 'Data', 'Boston', 'Public']": 100,
 "['Demography', 'Class', 'Work', 'Education', 'Boston', '1880-1930', 'Harvard', 'Dataverse', 'zip', 'vnd.openxmlformats-officedocument.spreadsheetml.sheet']": 43,
 "['Non', 'Public', 'Schools', 'Analyze', 'Boston', 'schools', 'public', 'open', 'non', 'data']": 59,
 "['Public', 'Schools',

In [330]:
import operator

sorted_x = sorted(fuzzy_scores.items(), key=operator.itemgetter(1), reverse=True)
sorted_x[0:3]

[("['Boston', 'Education', 'Data', 'BuildBPS', ':', 'Facilities', 'Educational', 'Data', 'Boston', 'Public']",
  100),
 ("['Non', 'Public', 'Schools', 'Analyze', 'Boston', 'schools', 'public', 'open', 'non', 'data']",
  59),
 ("['Demography', 'Class', 'Work', 'Education', 'Boston', '1880-1930', 'Harvard', 'Dataverse', 'zip', 'vnd.openxmlformats-officedocument.spreadsheetml.sheet']",
  43)]

**Fuzzy text string matching algorithm experiments**

In [27]:
from fuzzywuzzy import fuzz
print(fuzz.ratio(text, 'Boston education data'))
print(fuzz.ratio(text, 'I like to swim'))

2

In [28]:
from fuzzywuzzy import fuzz
print(fuzz.partial_ratio(text, 'Boston education data'))
print(fuzz.partial_ratio(text, 'I like to swim'))

36

In [29]:
#Sometimes a good option - resilient to false positives 
from fuzzywuzzy import fuzz
print(fuzz.token_sort_ratio(text, 'Boston education data'))
print(fuzz.token_sort_ratio(text, 'I like to swim'))

2

In [30]:
print(fuzz.token_set_ratio(text, 'Boston education data'))
print(fuzz.token_set_ratio(text, 'I like to swim'))

25